# Validação dos dados

## Introdução

Trata-se da validação dos dados dos boletins diários sobre infecções pela Covid-19, publicados pela Prefeitura Municipal de Santo Antônio da Platina em seu site <https://www.santoantoniodaplatina.pr.gov.br/>. 

O primeiro boletim foi publicado em 15/04/2020. 

Foram adotados pelos menos três modelos de boletins para divulgações das informações.

O primeiro modelo continha informações sobre: casos confirmados, descartados, sem coleta e em investigação. 
Por sua vez, esses atributos eram subdivididos em pelo menos três diferentes tipos de definições de casos: casos suspeitos com exames coletados; casos suspeitos Síndrome Gripal sem coleta de exames (Isolamento domiciliar); casos suspeitos pacientes internados do próprio Município.

Esse modelo foi utilizado pelo menos entre as datas 15/04/2020 até 01/06/2020. Ressaltasse que nesse período os boletins não eram rigorosamente diários.

Um segundo modelo foi adotado, pelo menos, entre 02/06/2020 até 02/07/2020.
Em um primeiro momento continha dados sobre: casos em investigação; descartados; confirmados; recuperados e óbitos. 
Em um segundo momento passou a incluir os casos ativos.

Nesses dois primeiros modelos também é apresentado o horário de atualização dos dados.

Em 03/07/2020 é adotado o modelo atual que apresenta os seguintes dados: descartados; em investigação; confirmados; examinados; recuperados; ativos; desse último subdivide os caso em hospital e domicílio; número de óbitos; casos examinados e novamente, em maior destaque, os casos confirmados.

Esse último modelo possui uma arte (com cores, diagramação e imagens de fundo) mais elaborada do que as versões anteriores. 
Pressupõe-se disso que o principal objetivo desse modelo de boletim é ser oferecer uma comunicação direta com o cidadão que poderia distribuir esses boletins através de suas próprias mídias sociais (Whatsapp, Facebook, etc.).
Tanto que a partir de 09/07/2020, esses boletins são disponibilizados em formatos de imagens com extensão .jpeg e não mais como arquivos na extensão .pdf.

Esses boletins também são disponibilizados como pop-up na página inicial da Prefeitura Municipal.

Entretanto, não foi possível localizar uma explicação mais detalhadas sobre os dados informados. 
Desta forma, para as validações que se seguem são feitas assumindo-se algumas condições, que são explicadas a seguir.
Por isso preferiu-se restringir aos dados apresentados apenas conforme um único modelo.
Uma tentativa de unir os dados poderia introduzir algum tipo de confusão e também de dados faltantes.

Os dados tratados a seguir se referem àqueles apresentados conforme o terceiro modelo do boletim, publicados entre os dias 03/07/2020 e 31/12/2020.

## Materiais e métodos

Os dados tratados a seguir são aqueles publicados entre os dias 03/07/2020 e 31/12/2020.

A coleta dos dados se deu de forma manual, uma vez que a maioria dos boletins é disponibilizada em formato de imagem (extesão jpeg) não permitindo a leitura de forma automática por máquina.
Não foi possível encontrar os dados em formatos abertos tais como, csv ou até mesmo por meio de planilhas eletrônicas.

A análise é executada utilizando a linguagem de Programação Python (versão 3.8) o software de gerenciamento de pacotes Anaconda (<https://anaconda.org/>) e as análises são feitas no ambiente Jupyter Notebook (<https://jupyter.org/>).

Foram utilizados as bibliotecas Numpy, Pandas, ```matplotlib``` e Seaborn nessa fase de análise. Maiores detalhes sobre outros pacotes utilizados podem ser encontradas no arquivo ```requeriments.txt```.

In [1]:
import os
from os import path
from datetime import datetime

import numpy as np
import pandas as pd

In [36]:
# import pandas_profiling

In [2]:
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
# definição do arquivo com a base de dados
filename = path.join('..', 'data', 'raw', 'BOLETIM_DIARIO_CORONAVIRUS_SAP.xlsx')

print(filename, path.exists(filename))

..\data\raw\BOLETIM_DIARIO_CORONAVIRUS_SAP.xlsx True


In [5]:
# Leitura da base de dados

# help(pd.read_excel)

frame = pd.read_excel(filename,
                   index_col="DATA",
                   parse_dates=['DATA'])

In [6]:
assert frame.index[0].month == 7, "Problema ao converter datas: o primeiro registro é de julho."
assert frame.index[-1].month == 12, "Problema ao converter datas: o último registro é de dezembro"
assert frame.index[0].day == 3, "Problema ao converter datas: primeiro registro deveria ser 03/07/2020"
assert frame.index[-1].day == 31, "Problema ao converter datas: o último registro é 31/12/2020"

In [7]:
assert frame.shape == (182, 9), "Não é do tamanho esperado: considerar somente os dados de 2020"

In [8]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 182 entries, 2020-07-03 to 2020-12-31
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   DESCARTADOS      182 non-null    int64
 1   EM INVESTIGACAO  182 non-null    int64
 2   CONFIRMADOS      182 non-null    int64
 3   EXAMINADOS       182 non-null    int64
 4   RECUPERADOS      182 non-null    int64
 5   ATIVOS           182 non-null    int64
 6   HOSPITAL         182 non-null    int64
 7   DOMICILIO        182 non-null    int64
 8   OBITOS           182 non-null    int64
dtypes: int64(9)
memory usage: 14.2 KB


In [27]:
frame.isnull().sum()

DESCARTADOS        0
EM INVESTIGACAO    0
CONFIRMADOS        0
EXAMINADOS         0
RECUPERADOS        0
ATIVOS             0
HOSPITAL           0
DOMICILIO          0
OBITOS             0
NAO_EXAMINADOS     0
dtype: int64

## Considerações sobre os atributos

Todos os atributos são numéricos e, por se tratarem de uma contagem, o domínio dos atributos vão ser o conjunto dos números naturais (0, 1, 2, 3...).

Alguns atributos são acumulativos isto é, no seu valor está condita a soma dos dias anteriores. Enquanto outros atributos representam o estado naquele dia.

Os atributos acumulativos são:

    DESCARTADOS
    CONFIRMADOS
    EXAMINADOS
    RECUPERADOS
    OBITOS
    
Atributos que representam um estado de determinado dia são:

    EM INVESTIGACAO
    ATIVOS
    HOSPITAL 
    DOMICILIO
    
    
Algumas relações entre esses atributos são esperadas, por exemplo:

1. Que o número casos ativos (ATIVOS) seja igual a soma dos atributos HOSPITAL e DOMICILIO, para determinada data;
2. Que o número de casos examinados (atributo EXAMINADOS) seja igual a soma dos valores em DESCARTADOS, CONFIRMADOS e EM INVESTIGACAO;
3. Que o valor do atributo CONFIRMADOS seja igual a soma dos atributos RECUPERADOS, OBTIOS, e ATIVOS;

Nos dois últimos itens é existe uma relação entre atributos cumulativos e atributos que representam um estado para determinado dia. A natureza desses atributos são diferentes e talvez a soma direta desses atributos pode não ser possível ou representar um resultado equivocado.

Porém o número de casos EM INVESTIGACAO possuem apenas dois resultados possíveis: DESCARTADOS ou CONFIRMADOS. Igualmente os casos enquadrados como ATIVOS, só terão dois resultados possíveis: RECUPERADOS e OBITOS.

In [9]:
frame.head()

,DESCARTADOS,EM INVESTIGACAO,CONFIRMADOS,EXAMINADOS,RECUPERADOS,ATIVOS,HOSPITAL,DOMICILIO,OBITOS
DATA,,,,,,,,,
2020-07-03,690,263,105,1058,63,38,3,35,4
2020-07-04,709,251,109,1069,65,43,3,40,4
2020-07-05,711,252,110,1073,71,35,3,32,4
2020-07-06,722,293,121,1136,79,38,2,36,4
2020-07-07,794,264,126,1184,85,37,4,33,4


In [10]:
assert all(frame['ATIVOS'] == frame['HOSPITAL'] + frame['DOMICILIO']), "O número de casos ativo deve ser a soma dos casos em DOMICILIO e HOSPITAL"

In [11]:
all(frame['EXAMINADOS'] == frame['DESCARTADOS'] + frame['CONFIRMADOS'] + frame['EM INVESTIGACAO'])

False

In [12]:
idx = frame['EXAMINADOS'] == (frame['DESCARTADOS'] + frame['CONFIRMADOS'] + frame['EM INVESTIGACAO'])

In [13]:
frame[~idx]

,DESCARTADOS,EM INVESTIGACAO,CONFIRMADOS,EXAMINADOS,RECUPERADOS,ATIVOS,HOSPITAL,DOMICILIO,OBITOS
DATA,,,,,,,,,
2020-09-08,2221,27,322,2569,306,9,0,9,7
2020-10-25,2690,21,364,3073,352,3,0,3,9


In [14]:
frame[~idx]['DESCARTADOS'] + frame[~idx]['CONFIRMADOS'] + frame[~idx]['EM INVESTIGACAO']

DATA
2020-09-08    2570
2020-10-25    3075
dtype: int64

In [15]:
frame[~idx]['EXAMINADOS']

DATA
2020-09-08    2569
2020-10-25    3073
Name: EXAMINADOS, dtype: int64

In [16]:
frame[~idx][['DESCARTADOS', 'CONFIRMADOS', 'EM INVESTIGACAO']]

,DESCARTADOS,CONFIRMADOS,EM INVESTIGACAO
DATA,,,
2020-09-08,2221,322,27
2020-10-25,2690,364,21


In [17]:
frame['NAO_EXAMINADOS'] = frame['EXAMINADOS'] - (frame['DESCARTADOS'] + frame['CONFIRMADOS'] + frame['EM INVESTIGACAO'])

In [18]:
all(frame[idx]['NAO_EXAMINADOS'] == 0)

True

In [19]:
frame[~idx]['NAO_EXAMINADOS']

DATA
2020-09-08   -1
2020-10-25   -2
Name: NAO_EXAMINADOS, dtype: int64

O número de confirmados deve ser igual a soma dos número de casos recuperados, do número de ativos no dia, e do número de óbitos.

In [20]:
all(frame['CONFIRMADOS'] == frame['RECUPERADOS'] + frame['ATIVOS'] + frame['OBITOS'])

False

In [21]:
idx = frame['CONFIRMADOS'] == frame['RECUPERADOS'] + frame['ATIVOS'] + frame['OBITOS']

frame.shape[0] - sum(idx)

1

In [22]:
frame[~idx]

,DESCARTADOS,EM INVESTIGACAO,CONFIRMADOS,EXAMINADOS,RECUPERADOS,ATIVOS,HOSPITAL,DOMICILIO,OBITOS,NAO_EXAMINADOS
DATA,,,,,,,,,,
2020-07-04,709,251,109,1069,65,43,3,40,4,0


In [23]:
print(frame[~idx][['RECUPERADOS', 'ATIVOS', 'OBITOS']].sum(axis=1))

frame[~idx][['RECUPERADOS', 'ATIVOS', 'OBITOS', 'CONFIRMADOS']]

DATA
2020-07-04    112
dtype: int64


,RECUPERADOS,ATIVOS,OBITOS,CONFIRMADOS
DATA,,,,
2020-07-04,65,43,4,109


In [24]:
frame.loc['07/04/2020':'07/10/2020'][['RECUPERADOS', 'ATIVOS', 'OBITOS', 'CONFIRMADOS']]

,RECUPERADOS,ATIVOS,OBITOS,CONFIRMADOS
DATA,,,,
2020-07-04,65,43,4,109
2020-07-05,71,35,4,110
2020-07-06,79,38,4,121
2020-07-07,85,37,4,126
2020-07-08,93,42,4,139
2020-07-09,98,38,4,140
2020-07-10,101,40,4,145


Os atributos cumulativos devem ser crescentes (igual ou superior ao dia anterior).

Os atributos acumulativos são:

    DESCARTADOS
    CONFIRMADOS
    EXAMINADOS
    RECUPERADOS
    OBITOS

In [32]:
frame["DESCARTADOS"].is_monotonic

False

In [33]:
frame['CONFIRMADOS'].is_monotonic

True

In [34]:
frame['EXAMINADOS'].is_monotonic

False

In [31]:
frame["RECUPERADOS"].is_monotonic

True

In [35]:
frame['OBITOS'].is_monotonic

True